In [16]:
import tabula
import pandas as pd
import re
import unicodedata
from pathlib import Path

sequenciaDeTabelas = ["10-11","12-13","14-28","29-38","39-55","56-71","72-85"]



LocalNT = Path().resolve()
caminhoPDF = LocalNT.parent /"data" / "raw" / "eOuve - Limeira.pdf"
tabelas = []



modeloSituacao = ['aberto',	'agendado',	'cancelado',	'concluido'	,'em_andamento',	'pendente',	'total']
modeloTipo = ['denuncia',	'doacao',	'elogio',	'informacao',	'reclamacao',	'simplifique'	,'solicitacao',	'sugestao',	'total']

def limparTexto(tabela,coluna):
    

    def limpezaDeCelula(texto):
        if (pd.isna(texto)):
            return texto
        texto =str(texto).replace('\r',' ')

        nfkdForm = unicodedata.normalize('NFKD',texto)
        textoSemAcento = nfkdForm.encode('ASCII','ignore').decode('ASCII')


        textoLimpo = re.sub(r'[^a-zA-Z0-9\s]','',textoSemAcento)
        textoLimpo = re.sub(r'\s+',' ',textoLimpo).strip()

        return textoLimpo

    tabela[coluna] = tabela[coluna].apply(limpezaDeCelula)
    return tabela
    

def limparNomeColuna(nomeColuna):
    nomeColuna = str(nomeColuna)

    nfkdForm = unicodedata.normalize('NFKD',nomeColuna)
    nomeLimpo = nfkdForm.encode('ASCII','ignore').decode('ASCII')

    nomeLimpo = nomeLimpo.lower()

    nomeLimpo = re.sub(r'[-–—\s/.]+','_',nomeLimpo)
    nomeLimpo = re.sub(r'[^a-z0-9_]','',nomeLimpo)
    nomeLimpo = re.sub(r'__','_',nomeLimpo).strip('_')


    return nomeLimpo







def limparValores(tabela,listaDeColunas):
    for coluna in listaDeColunas:
        tabela[coluna] = tabela[coluna].astype(str).str.replace('.','',regex = False).str.replace(',','.',regex = False)
        tabela[coluna] = pd.to_numeric(tabela[coluna], errors='coerce')
        tabela[coluna] = tabela[coluna].fillna(0).astype('Int64')

    return tabela



#para evitar a criacao de outra lista de colunas sem o parametro Total, o metodo abaixo tem a variavel colunasParaSomar que recebe todas as colunas -1 que é a Total
def calcularTotal(tabela,modeloColunas):
    colunasParaSomar = [limparNomeColuna(col) for col in modeloColunas[:-1]]
    tabela['total'] = tabela[colunasParaSomar].sum(axis=1)
    return tabela





for sequencia in sequenciaDeTabelas:

  tabela = tabula.read_pdf(
        caminhoPDF,
        pages=sequencia,
        lattice=True,
        multiple_tables=False,
        encoding = "latin1"
        
        
        )[0];

  tabela.columns = [limparNomeColuna(col) for col in tabela.columns]
  tabela.drop(tabela.tail(1).index,inplace=True)
  tabelas.append(tabela)


tabela1 = tabelas[0]
tabela2 = tabelas[1]
tabela3 = tabelas[2]
tabela4 = tabelas[3]
tabela5 = tabelas[4]
tabela6 = tabelas[5]
tabela7 = tabelas[6]







valoresSituacao = {'Aberto' : int,	'Agendado' : int,	'Cancelado': int,	'Concluído': int	,'Em andamento': int,	'Pendente': int,'Total' : int}
valoresTipo = {'Denúncia': int,	'Doação': int,	'Elogio': int,	'Informação': int,	'Reclamação': int,	'Simplifique': int	,'Solicitação': int,	'Sugestão': int,'Total' : int}







tabela1 = limparValores(tabela1,modeloTipo)
tabela2 = limparValores(tabela2,modeloSituacao)
tabela3 = limparValores(tabela3,modeloTipo)
tabela4 = limparValores(tabela4,modeloSituacao)
tabela5 = limparValores(tabela5,modeloTipo)
tabela6 = limparValores(tabela6,modeloSituacao)
tabela7 = limparValores(tabela7,modeloTipo)


tabela1 = calcularTotal(tabela1,modeloTipo)
tabela2 = calcularTotal(tabela2,modeloSituacao)
tabela3 = calcularTotal(tabela3,modeloTipo)
tabela4 = calcularTotal(tabela4,modeloSituacao)
tabela5 = calcularTotal(tabela5,modeloTipo)
tabela6 = calcularTotal(tabela6,modeloSituacao)
tabela7 = calcularTotal(tabela7,modeloTipo)



tabela1 = limparTexto(tabela1,'secretaria')
tabela2 = limparTexto(tabela2,'secretaria')
tabela3 = limparTexto(tabela3,'assunto')
tabela4 = limparTexto(tabela4,'assunto')
tabela5 = limparTexto(tabela5,'bairro')
tabela6 = limparTexto(tabela6,'bairro')
tabela7 = limparTexto(tabela7,'secretaria')

#o codigo abaixo indica que praticamente todo o conteudo foi lido como String(object)
#display(tabela1.dtypes)
#display(tabela2.dtypes) 
#display(tabela3.dtypes) 
#display(tabela4.dtypes) 
#display(tabela5.dtypes) 
#display(tabela6.dtypes) 
#display(tabela7.dtypes) 


listaDeTabelas = [tabela1,tabela2,tabela3,tabela4,tabela5,tabela6,tabela7]

def tabelasParaCsv(listacomTabelas):
    n = 1
    for tabela in listacomTabelas:
        tabela.to_csv(f'Tabela{n}_TratadaHackaton.csv',mode = 'w',index = False, header = True)
        n = n + 1

tabelasParaCsv(listaDeTabelas)



display(tabela1)
display(tabela2)
display(tabela3)
display(tabela4)
display(tabela5)
display(tabela6)
display(tabela7)




,secretaria,denuncia,doacao,elogio,informacao,reclamacao,simplifique,solicitacao,sugestao,total
0,Animais,0,0,0,0,0,0,0,0,0
1,Denuncia Descarte Irregular,172,0,0,0,0,0,150,0,322
2,Direitos Humanos Denuncias de discriminacao e ...,1,0,0,0,1,0,0,0,2
3,LGPD Lei Geral de Protecao de Dados,17,0,0,0,0,0,30,0,47
4,PAT Posto de Atendimento do Trabalhador,0,0,0,0,0,0,0,0,0
5,Secretaria de Agronegocios e Areas Rurais,2,0,0,0,1,0,326,2,331
6,Secretaria de Comunicacao,44,0,15,10267,115,0,24,37,10502
7,Secretaria de Esportes e Lazer,0,0,0,0,1,0,3,95,99
8,Secretaria de Fazenda,43,0,0,0,10,0,494,3,550
9,Secretaria de Meio Ambiente e Saneamento,139,0,0,0,4,0,468,3,614


,secretaria,aberto,agendado,cancelado,concluido,em_andamento,pendente,total
0,Animais,0,0,0,0,0,0,0
1,Denuncia Descarte Irregular,61,0,0,227,33,1,322
2,Direitos Humanos Denuncias de discriminacao e ...,0,0,0,2,0,0,2
3,LGPD Lei Geral de Protecao de Dados,0,0,40,7,0,0,47
4,PAT Posto de Atendimento do Trabalhador,0,0,0,0,0,0,0
5,Secretaria de Agronegocios e Areas Rurais,2,0,65,179,84,1,331
6,Secretaria de Comunicacao,1,0,15,10482,4,0,10502
7,Secretaria de Esportes e Lazer,4,0,1,93,1,0,99
8,Secretaria de Fazenda,37,0,4,509,0,0,550
9,Secretaria de Meio Ambiente e Saneamento,121,0,2,452,37,2,614


,assunto,denuncia,doacao,elogio,informacao,reclamacao,simplifique,solicitacao,sugestao,total
0,Animais Outros,0,0,0,0,0,0,0,0,0
1,Denuncia Descarte Irregular DESCARTE IRREGULAR...,172,0,0,0,0,0,150,0,322
2,Denuncia Descarte Irregular Outros,0,0,0,0,0,0,0,0,0
3,Direitos Humanos Denuncias de discriminacao e ...,0,0,0,0,0,0,0,0,0
4,Direitos Humanos Denuncias de discriminacao e ...,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
132,Secretaria dos Transportes e Mobilidade Urbana...,9,0,2,0,9,0,12,449,481
133,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,0,0,0,59,1,60
134,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,0,0,0,20,0,20
135,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,0,0,0,26,1,27


,assunto,aberto,agendado,cancelado,concluido,em_andamento,pendente,total
0,Animais Outros,0,0,0,0,0,0,0
1,Denuncia Descarte Irregular DESCARTE IRREGULAR...,61,0,0,227,33,1,322
2,Denuncia Descarte Irregular Outros,0,0,0,0,0,0,0
3,Direitos Humanos Denuncias de discriminacao e ...,0,0,0,0,0,0,0
4,Direitos Humanos Denuncias de discriminacao e ...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
136,Secretaria dos Transportes e Mobilidade Urbana...,1,0,1,422,50,7,481
137,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,54,5,1,60
138,Secretaria dos Transportes e Mobilidade Urbana...,1,0,0,17,2,0,20
139,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,27,0,0,27


,bairro,denuncia,doacao,elogio,informacao,reclamacao,simplifique,solicitacao,sugestao,total
0,Anavec,19,0,0,0,1,0,203,4,227
1,Area Rural de Limeira,29,0,0,1,1,0,301,4,336
2,Bairro da Geada,9,0,0,1,0,0,21,0,31
3,Bairro dos Frades,0,0,0,0,0,0,9,0,9
4,Bairro dos Loiolas,0,0,0,0,2,0,38,0,40
...,...,...,...,...,...,...,...,...,...,...
262,Vila Sao Joao,23,0,0,0,1,0,206,6,236
263,Vila Sao Luiz,4,0,0,0,3,0,148,5,160
264,Vila Sao Roque,1,0,0,0,0,0,81,0,82
265,Vila Solar,0,0,0,0,0,0,35,0,35


,bairro,aberto,agendado,cancelado,concluido,em_andamento,pendente,total
0,Anavec,8,0,3,192,23,1,227
1,Area Rural de Limeira,19,0,44,203,69,1,336
2,Bairro da Geada,1,0,0,26,4,0,31
3,Bairro dos Frades,0,0,2,6,1,0,9
4,Bairro dos Loiolas,13,0,3,18,6,0,40
...,...,...,...,...,...,...,...,...
264,Vila Sao Joao,42,0,2,162,30,0,236
265,Vila Sao Luiz,14,0,2,130,14,0,160
266,Vila Sao Roque,5,0,1,75,1,0,82
267,Vila Solar,0,0,1,30,4,0,35


,secretaria,denuncia,doacao,elogio,informacao,reclamacao,simplifique,solicitacao,sugestao,total
0,Animais Outros,0,0,0,0,0,0,0,0,0
1,Denuncia Descarte Irregular DESCARTE IRREGULAR...,27,0,0,0,0,0,23,0,50
2,Denuncia Descarte Irregular Outros,0,0,0,0,0,0,0,0,0
3,Direitos Humanos Denuncias de discriminacao e ...,0,0,0,0,0,0,0,0,0
4,Direitos Humanos Denuncias de discriminacao e ...,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
135,Secretaria dos Transportes e Mobilidade Urbana...,2,0,0,0,3,0,1,24,30
136,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,0,0,0,1,0,1
137,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,0,0,0,5,0,5
138,Secretaria dos Transportes e Mobilidade Urbana...,0,0,0,0,0,0,2,0,2
